In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, CategoricalNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

import spacy
import re
import numpy as np
import matplotlib.pyplot as plt
import emoji

from unicodedata import normalize
from tqdm import tqdm
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from datasets import list_datasets, load_dataset

In [4]:
def preprocessar(tweets):
    repetion_pattern = re.compile(r'(.)\1\1+')
    new_tweets = []
    with tqdm(total=len(tweets), colour='green', desc='Processando') as pbar:
      for tweet in tweets:
          tweet = emoji.demojize(tweet, language='pt')
          tweet = tweet.replace('_', ' ')
          tweet = normalize('NFKD', tweet).encode('ASCII', 'ignore').decode('ASCII')
          tweet = repetion_pattern.sub(r'\1', tweet)
          tweet = re.sub(r'https?://\w+', '', tweet)
          tweet = re.sub(r'@\w+', ' ', tweet)
          tweet = re.sub(r'\s\s+', ' ', tweet)
          new_tweets.append(tweet.strip())
          pbar.update(1)
    return new_tweets

In [24]:
# Carregar os dados
data = pd.read_csv('./dataset-merged/dataset_merged.csv')  # Seu arquivo de dados

# Dividir os dados em treinamento, teste
texts = data["tweet"].to_list()
labels = data["hatespeech"].to_list()

X_train, X_test, y_train, y_test = train_test_split(preprocessar(texts), labels, test_size=0.33, random_state=42)

Processando: 100%|█████████████████████████████████████████████████████████████| 12622/12622 [00:01<00:00, 9719.03it/s]


In [25]:
#vectorizer = CountVectorizer(binary=True, max_features=None, ngram_range=(1, 1))
#vectorizer = CountVectorizer(binary=False, max_features=None, ngram_range=(1, 1))
vectorizer = TfidfVectorizer(max_features=None, ngram_range=(1, 1))

vectorizer.fit(X_train)

print(f'Vocabulary: {len(vectorizer.vocabulary_)}')
X_train = vectorizer.transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()

Vocabulary: 19193


In [26]:
classifiers = {
    'Logistic Regression': LogisticRegression(class_weight=None, max_iter=1500),
    'Multinomial NB': MultinomialNB(),
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(class_weight=None),
    'XGBClassifier': XGBClassifier(n_estimators=100),
    'Random Forest': RandomForestClassifier(n_estimators=100, class_weight=None)
}

In [27]:
print('\nEvaluation\n')

for clf_name, clf in classifiers.items():   
    print(f'\n  Classifier: {clf_name}\n')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    report = classification_report(y_test, y_pred, zero_division=0)
    print(report)


Evaluation


  Classifier: Logistic Regression

              precision    recall  f1-score   support

           0       0.76      0.62      0.69      1596
           1       0.79      0.88      0.83      2570

    accuracy                           0.78      4166
   macro avg       0.78      0.75      0.76      4166
weighted avg       0.78      0.78      0.78      4166


  Classifier: Multinomial NB

              precision    recall  f1-score   support

           0       0.85      0.26      0.40      1596
           1       0.68      0.97      0.80      2570

    accuracy                           0.70      4166
   macro avg       0.77      0.62      0.60      4166
weighted avg       0.75      0.70      0.65      4166


  Classifier: KNN

              precision    recall  f1-score   support

           0       0.62      0.52      0.56      1596
           1       0.73      0.80      0.76      2570

    accuracy                           0.69      4166
   macro avg       0.67     